In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

In [3]:
file1 = open("../../data/features/2019_05_17_TSLA_y_08_yishuang.pickle",'rb')
tsla = pickle.load(file1, encoding='utf-8')

file2 = open("../../data/features/2019_05_16_all_features.pickle",'rb')
elon = pickle.load(file2, encoding='utf-8')

In [4]:
elon.columns

Index(['id', 'Datetime', 'text', 'sentiment', 'subjectivity', 'isreply',
       'year', 'month', 'day', 'date', 'DayofWeek', 'Weekend', 'Time',
       'BinaryTrading', 'DayDistance', 'gtrend', 'CommentSentimental',
       'CommentSubjectivity', 'countComment', 'posCommentSum', 'negCommentSum',
       'CommentSD', 'posCommentSD', 'negCommentSD', 'negCommentPercent',
       'posCommentPercent'],
      dtype='object')

In [5]:
tsla.columns

Index(['Hour2', 'Hour3', 'Hour4', 'Hour5', 'Hour6', 'Hour7', 'Date',
       'Daily_volume', 'Datetime', 'Hour_volume', 'Trading_Hour',
       'Roll_volume', 'Moving_average_5', 'Volume_perc', 'Perc_shift_7',
       'Perc_shift_14', 'Perc_shift_21', 'Perc_shift_28', 'Perc_shift_35',
       'Perc_shift_42', 'Perc_shift_49', 'AR_pred', 'Residual'],
      dtype='object')

In [6]:
# Add start and end datetime
tsla["Start_Datetime"] = tsla["Datetime"]
tsla['End_Datetime'] = tsla['Datetime'] + pd.DateOffset(hours=1)

In [7]:
tsla.head()

,Hour2,Hour3,Hour4,Hour5,Hour6,Hour7,Date,Daily_volume,Datetime,Hour_volume,...,Perc_shift_14,Perc_shift_21,Perc_shift_28,Perc_shift_35,Perc_shift_42,Perc_shift_49,AR_pred,Residual,Start_Datetime,End_Datetime
0,0,0,0,0,0,0,2016-06-06,2249500,2016-06-06 13:30:00+00:00,802995.0,...,0.191230,0.497019,0.129291,0.288979,0.333049,0.250512,0.180036,0.209508,2016-06-06 13:30:00+00:00,2016-06-06 14:30:00+00:00
1,1,0,0,0,0,0,2016-06-06,2249500,2016-06-06 14:30:00+00:00,279458.0,...,0.121435,0.274092,0.093569,0.162169,0.276517,0.241683,0.174876,-0.043552,2016-06-06 14:30:00+00:00,2016-06-06 15:30:00+00:00
2,0,1,0,0,0,0,2016-06-06,2249500,2016-06-06 15:30:00+00:00,292325.0,...,0.082991,0.140525,0.074788,0.078377,0.151645,0.095730,0.113209,0.018052,2016-06-06 15:30:00+00:00,2016-06-06 16:30:00+00:00
3,0,0,1,0,0,0,2016-06-06,2249500,2016-06-06 16:30:00+00:00,170419.0,...,0.086119,0.081911,0.066586,0.059976,0.075488,0.104234,0.087077,-0.012592,2016-06-06 16:30:00+00:00,2016-06-06 17:30:00+00:00
4,0,0,0,1,0,0,2016-06-06,2249500,2016-06-06 17:30:00+00:00,214303.0,...,0.091694,0.069936,0.078209,0.090399,0.085611,0.157451,0.097929,-0.006690,2016-06-06 17:30:00+00:00,2016-06-06 18:30:00+00:00


In [8]:
tsla["Start_Datetime"] = pd.to_datetime(tsla["Start_Datetime"], utc = True)
tsla["End_Datetime"] = pd.to_datetime(tsla["End_Datetime"], utc = True)
elon["Datetime"] = pd.to_datetime(elon["Datetime"], utc = True)

In [9]:
elon_new = elon[(elon['Datetime'] >= tsla.iloc[0]['Start_Datetime']) & (elon['Datetime'] <= tsla.iloc[-1]['End_Datetime'])]

In [10]:
elon_new.tail()

,id,Datetime,text,sentiment,subjectivity,isreply,year,month,day,date,...,CommentSentimental,CommentSubjectivity,countComment,posCommentSum,negCommentSum,CommentSD,posCommentSD,negCommentSD,negCommentPercent,posCommentPercent
768,744549842496655360,2016-06-19 15:18:00+00:00,tesla owner drives through a flooded tunnel & ...,-0.125,0.375,0,2016,6,19,2016-06-19,...,10.792272,25.691865,83,12.846161,-2.053889,0.279118,0.241532,0.238588,0.108434,0.397590
769,742120520678703105,2016-06-12 22:24:00+00:00,highly recommend anything by sean carrollhttps...,0.160,0.540,0,2016,6,12,2016-06-12,...,5.023125,8.237708,23,5.281458,-0.258333,0.260117,0.219455,0.135529,0.086957,0.652174
770,741410425787404288,2016-06-10 23:23:00+00:00,nhtsa confirmed today that they found no safet...,0.200,0.750,0,2016,6,10,2016-06-10,...,8.903007,17.389161,45,9.891896,-0.988889,0.297564,0.249481,0.176471,0.088889,0.577778
771,741315106370244609,2016-06-10 17:04:00+00:00,nice comment about the model s by cvk on @ top...,0.600,1.000,0,2016,6,10,2016-06-10,...,4.225000,5.852778,19,4.375000,-0.150000,0.287819,0.234299,0.000000,0.052632,0.526316
772,740383850820296707,2016-06-08 03:24:00+00:00,would like to clarify that tesla is working ex...,0.000,0.000,0,2016,6,8,2016-06-08,...,11.134578,26.082522,117,12.780411,-1.645833,0.241406,0.269885,0.270615,0.059829,0.324786


In [ ]:
tweet_datetime = elon_new.iloc[0]['Datetime']
subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
subset_df

In [11]:
# y_target: volume percentage
volume_percs = []
trading_hour = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume percentage of first row of subset df
    perc = subset_df.iloc[0]['Volume_perc']
    hour = subset_df.iloc[0]['Trading_Hour']
    
    # append to list
    volume_percs.append(perc)
    trading_hour.append(int(hour))

elon_new['Volume_perc'] = pd.Series(volume_percs)
elon_new['Trading_Hour'] = pd.Series(trading_hour)

In [12]:
elon_new.head()

,id,Datetime,text,sentiment,subjectivity,isreply,year,month,day,date,...,countComment,posCommentSum,negCommentSum,CommentSD,posCommentSD,negCommentSD,negCommentPercent,posCommentPercent,Volume_perc,Trading_Hour
27,1112847351595466753,2019-04-01 22:40:00+00:00,that car seemed so advanced when i watched tha...,0.250000,0.300000,1,2019,4,1,2019-04-01,...,25,5.412311,0.000000,0.292040,0.265866,0.000000,0.000000,0.480000,0.263361,1.0
28,1112839897218310144,2019-04-01 22:11:00+00:00,"going forward, all tesla stores will be tiny &...",0.000000,0.500000,1,2019,4,1,2019-04-01,...,200,20.584905,-4.033257,0.223683,0.241346,0.118763,0.125000,0.315000,0.124610,5.0
29,1112123741574586368,2019-03-30 22:45:00+00:00,we’ve been so mired in production & logistics ...,-0.025000,0.225000,0,2019,3,30,2019-03-30,...,1635,181.046184,-36.849299,0.244820,0.238605,0.224317,0.092355,0.304587,0.225978,1.0
30,1112106548363292672,2019-03-30 21:37:00+00:00,tesla semi delivering tesla carspic.twitter.co...,0.000000,0.000000,0,2019,3,30,2019-03-30,...,998,115.902111,-21.898603,0.253292,0.242533,0.240379,0.081162,0.301603,0.084750,7.0
31,1112036985504317442,2019-03-30 17:00:00+00:00,"amazing work by tesla delivery teams, especial...",0.182292,0.608333,0,2019,3,30,2019-03-30,...,321,44.390798,-8.188405,0.269115,0.258929,0.184834,0.112150,0.383178,0.137876,5.0


In [13]:
elon_new.groupby('Trading_Hour')['Trading_Hour'].count()

Trading_Hour
1.0    533
2.0     23
3.0     17
4.0     40
5.0     40
6.0     36
7.0     30
Name: Trading_Hour, dtype: int64

In [15]:
tsla.head()

,Hour2,Hour3,Hour4,Hour5,Hour6,Hour7,Date,Daily_volume,Datetime,Hour_volume,...,Perc_shift_14,Perc_shift_21,Perc_shift_28,Perc_shift_35,Perc_shift_42,Perc_shift_49,AR_pred,Residual,Start_Datetime,End_Datetime
0,0,0,0,0,0,0,2016-06-06,2249500,2016-06-06 13:30:00+00:00,802995.0,...,0.191230,0.497019,0.129291,0.288979,0.333049,0.250512,0.180036,0.209508,2016-06-06 13:30:00+00:00,2016-06-06 14:30:00+00:00
1,1,0,0,0,0,0,2016-06-06,2249500,2016-06-06 14:30:00+00:00,279458.0,...,0.121435,0.274092,0.093569,0.162169,0.276517,0.241683,0.174876,-0.043552,2016-06-06 14:30:00+00:00,2016-06-06 15:30:00+00:00
2,0,1,0,0,0,0,2016-06-06,2249500,2016-06-06 15:30:00+00:00,292325.0,...,0.082991,0.140525,0.074788,0.078377,0.151645,0.095730,0.113209,0.018052,2016-06-06 15:30:00+00:00,2016-06-06 16:30:00+00:00
3,0,0,1,0,0,0,2016-06-06,2249500,2016-06-06 16:30:00+00:00,170419.0,...,0.086119,0.081911,0.066586,0.059976,0.075488,0.104234,0.087077,-0.012592,2016-06-06 16:30:00+00:00,2016-06-06 17:30:00+00:00
4,0,0,0,1,0,0,2016-06-06,2249500,2016-06-06 17:30:00+00:00,214303.0,...,0.091694,0.069936,0.078209,0.090399,0.085611,0.157451,0.097929,-0.006690,2016-06-06 17:30:00+00:00,2016-06-06 18:30:00+00:00


In [16]:
# adding AR-y
ar_y = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row of subset df
    y = subset_df.iloc[0]['AR_pred']
    

    # append to list
    ar_y.append(y)

elon_new['AR_pred'] = pd.Series(ar_y)

In [17]:
elon_new

,id,Datetime,text,sentiment,subjectivity,isreply,year,month,day,date,...,posCommentSum,negCommentSum,CommentSD,posCommentSD,negCommentSD,negCommentPercent,posCommentPercent,Volume_perc,Trading_Hour,AR_pred
27,1112847351595466753,2019-04-01 22:40:00+00:00,that car seemed so advanced when i watched tha...,0.250000,0.300000,1,2019,4,1,2019-04-01,...,5.412311,0.000000,0.292040,0.265866,0.000000,0.000000,0.480000,0.263361,1.0,0.234082
28,1112839897218310144,2019-04-01 22:11:00+00:00,"going forward, all tesla stores will be tiny &...",0.000000,0.500000,1,2019,4,1,2019-04-01,...,20.584905,-4.033257,0.223683,0.241346,0.118763,0.125000,0.315000,0.124610,5.0,0.094620
29,1112123741574586368,2019-03-30 22:45:00+00:00,we’ve been so mired in production & logistics ...,-0.025000,0.225000,0,2019,3,30,2019-03-30,...,181.046184,-36.849299,0.244820,0.238605,0.224317,0.092355,0.304587,0.225978,1.0,0.246139
30,1112106548363292672,2019-03-30 21:37:00+00:00,tesla semi delivering tesla carspic.twitter.co...,0.000000,0.000000,0,2019,3,30,2019-03-30,...,115.902111,-21.898603,0.253292,0.242533,0.240379,0.081162,0.301603,0.084750,7.0,0.110576
31,1112036985504317442,2019-03-30 17:00:00+00:00,"amazing work by tesla delivery teams, especial...",0.182292,0.608333,0,2019,3,30,2019-03-30,...,44.390798,-8.188405,0.269115,0.258929,0.184834,0.112150,0.383178,0.137876,5.0,0.089315
32,1111788218142216192,2019-03-30 00:32:00+00:00,retrofits will start when our software is able...,0.295833,0.419792,1,2019,3,30,2019-03-30,...,60.349110,-9.847427,0.229875,0.211034,0.215596,0.106122,0.420408,0.294701,1.0,0.285169
33,1111762533520531456,2019-03-29 22:50:00+00:00,anyone who purchased full self-driving will ge...,0.155000,0.461667,0,2019,3,29,2019-03-29,...,40.492307,-8.223304,0.223112,0.201338,0.215351,0.111421,0.398329,0.065214,6.0,0.102730
34,1111001632731389952,2019-03-27 20:26:00+00:00,super proud of tesla autopilot team!,0.666667,0.833333,1,2019,3,27,2019-03-27,...,17.803204,-1.025000,0.298376,0.279697,0.086796,0.093023,0.500000,0.090368,7.0,0.117407
35,1110632394611015681,2019-03-26 19:59:00+00:00,i think so. for sure worth giving it our best ...,0.233333,0.359877,1,2019,3,26,2019-03-26,...,90.909926,-21.583804,0.221554,0.191146,0.223208,0.119952,0.382423,0.299102,1.0,0.259531
36,1110576683411628032,2019-03-26 16:18:00+00:00,hard to appreciate how much tesla autopilot im...,-0.045833,0.370833,1,2019,3,26,2019-03-26,...,43.359616,-7.845460,0.238042,0.190385,0.201954,0.134868,0.476974,0.299102,1.0,0.259531


In [20]:
elon_new.dropna(inplace = True)

In [21]:
with open('../../data/2019_05_17_ar_pred.pickle', 'wb') as handle:
        pickle.dump(elon_new, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# y_target 2: volume residual
residuals = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row of subset df
    residual = subset_df.iloc[0]['Residual']
    

    # append to list
    residuals.append(residual)

elon_new['Residual'] = pd.Series(residuals)

In [ ]:
with open('../../data/features/2019_05_17_features_finally.pickle', 'wb') as handle:
        pickle.dump(elon_new, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# rolling average volume percentage
roll_vol_perc = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume of first row of subset df
    vol_perc = subset_df.iloc[0]['Rolling_perc_volume']
 
    # append to list
    roll_vol_perc.append(vol_perc)

elon_new['Rolling_volume_perc'] = pd.Series(roll_vol_perc)

In [ ]:
# rolling average volume 
roll_vol = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume of first row of subset df
    vol = subset_df.iloc[0]['Rolling_volume']
 
    # append to list
    roll_vol.append(vol)

elon_new['Rolling_volume'] = pd.Series(roll_vol)

In [ ]:
# y_target 1: volume
volumes = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume of first row of subset df
    volume = subset_df.iloc[0]['Volume']
 
    # append to list
    volumes.append(volume)

elon_new['That_hour_volume'] = pd.Series(volumes)

In [ ]:
# y_target 2: volume percentage
volume_percs = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume percentage of first row of subset df
    perc = subset_df.iloc[0]['Percentage_Volume']

    # append to list
    volume_percs.append(perc)

elon_new['That_hour_volume_perc'] = pd.Series(volume_percs)

In [ ]:
# y_target 3: volume percentage residual
residual_perc = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row of subset df
    residual = subset_df.iloc[0]['Res_Perc_vol']
    

    # append to list
    residual_perc.append(residual)

elon_new['That_hour_res_vol_perc'] = pd.Series(residual_perc)

In [ ]:
# y_target 4: volume residual
residuals = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row of subset df
    residual = subset_df.iloc[0]['Res_vol']
    

    # append to list
    residuals.append(residual)

elon_new['That_hour_res_vol'] = pd.Series(residuals)

In [ ]:
elon_new.drop(['Datetime_after_1h'], axis = 1, inplace = True)

In [ ]:
elon_new.head(10)

In [ ]:
# y_target 5: volume in 2 hours
volumes = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume of first row and second row of subset df, then take average
    volume = (subset_df.iloc[0]['Volume'] + subset_df.iloc[1]['Volume'])/2
 
    # append to list
    volumes.append(volume)

elon_new['Volume_in_2h'] = pd.Series(volumes)

In [ ]:
# y_target 6: volume percentage in 2 hours
volume_percs = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume percentage of first row and second row of subset df, then take average
    perc = (subset_df.iloc[0]['Percentage_Volume'] + subset_df.iloc[1]['Percentage_Volume'])/2

    # append to list
    volume_percs.append(perc)

elon_new['Volume_perc_in_2h'] = pd.Series(volume_percs)

In [ ]:
# y_target 7: volume percentage residual in 2 hours
residual_perc = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row and second row of subset df and take average
    residual = (subset_df.iloc[0]['Res_Perc_vol'] + subset_df.iloc[0]['Res_Perc_vol'])/2

    # append to list
    residual_perc.append(residual)

elon_new['Res_vol_perc_in_2h'] = pd.Series(residual_perc)

In [ ]:
# y_target 8: volume residual in 2 hours
residuals = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row and second row of subset df and take average
    residual = (subset_df.iloc[0]['Res_vol'] + subset_df.iloc[0]['Res_vol'])/2
    

    # append to list
    residuals.append(residual)

elon_new['Res_vol_in_2h'] = pd.Series(residuals)

In [ ]:
elon_new.head()

In [ ]:
with open('../../data/features/2019_04_30_features_with_8y.pickle', 'wb') as handle:
        pickle.dump(elon_new, handle, protocol=pickle.HIGHEST_PROTOCOL)